In [ ]:
import sys
sys.path.append('E:/zlab/')
from loader import Loader, XModel
# ------------------------------------
import time
from sklearn.model_selection import train_test_split

import tables as tb
import mxnet as mx
from mxnet import nd, autograd, gluon
import gluonbook as gb
from mxnet import gluon, init, nd
from mxnet.gluon import data as gdata, loss as gloss, model_zoo

h5 = tb.open_file('E:/xdata/X.h5')

data = h5.root.cifar10

X = data.trainX[:]
y = data.trainY[:]
X_train, X_unlabel, y_train, y__unlabel = train_test_split(
    X, y, test_size=0.98, random_state=42)

batch_size = 32
trainset = Loader(X_train, y_train, batch_size, shuffle=True)
testset = Loader(data.testX, data.testY, batch_size, shuffle=False)

# ------------
num_epochs = 1
learning_rate = 0.01
pretrained_net = model_zoo.vision.resnet50_v2(pretrained=True)
finetune_net = model_zoo.vision.resnet50_v2(classes=10)
finetune_net.features = pretrained_net.features
finetune_net.output.initialize(init.Xavier())

In [ ]:
learning_rate = 0.1
batch_size = 40
epochs = 100
model = XModel(trainset, testset, finetune_net)
model.train(learning_rate, batch_size, epochs)

In [ ]:
from loader import aug_dataset

aug = aug_dataset()

X_ = nd.stack(*[aug.test(nd.array(x)) for x in X_unlabel])
X_ = X_.as_in_context(mx.gpu(0))

y_ = model.net.features(X_)

In [ ]:
y_.argmax(axis=1)

In [ ]:
y_.argmax(axis=1)

In [ ]:
features = X_unlabel.reshape((-1, 32*32*3)).astype(np.float32)

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

from sklearn import datasets
from sklearn.semi_supervised import label_propagation
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from itertools import cycle
from time import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import Birch, MiniBatchKMeans

In [ ]:
# Use all colors that matplotlib provides by default.
colors_ = cycle(colors.cnames.keys())

fig = plt.figure(figsize=(12, 4))
fig.subplots_adjust(left=0.04, right=0.98, bottom=0.1, top=0.9)

# Compute clustering with Birch with and without the final clustering step
# and plot.
birch_models = [Birch(threshold=1.7, n_clusters=None),
                Birch(threshold=1.7, n_clusters=10)]
final_step = ['without global clustering', 'with global clustering']

for ind, (birch_model, info) in enumerate(zip(birch_models, final_step)):
    t = time()
    birch_model.fit(features)
    time_ = time() - t
    print("Birch %s as the final step took %0.2f seconds" % (
          info, (time() - t)))

    # Plot result
    labels = birch_model.labels_
    centroids = birch_model.subcluster_centers_
    n_clusters = np.unique(labels).size
    print("n_clusters : %d" % n_clusters)

    ax = fig.add_subplot(1, 3, ind + 1)
    for this_centroid, k, col in zip(centroids, range(n_clusters), colors_):
        mask = labels == k
        ax.scatter(X[mask, 0], X[mask, 1],
                   c='w', edgecolor=col, marker='.', alpha=0.5)
        if birch_model.n_clusters is None:
            ax.scatter(this_centroid[0], this_centroid[1], marker='+',
                       c='k', s=25)
    ax.set_ylim([-25, 25])
    ax.set_xlim([-25, 25])
    ax.set_autoscaley_on(False)
    ax.set_title('Birch %s' % info)

# Compute clustering with MiniBatchKMeans.
mbk = MiniBatchKMeans(init='k-means++', n_clusters=10, batch_size=100,
                      n_init=10, max_no_improvement=10, verbose=0,
                      random_state=0)
t0 = time()
mbk.fit(features)
t_mini_batch = time() - t0
print("Time taken to run MiniBatchKMeans %0.2f seconds" % t_mini_batch)
mbk_means_labels_unique = np.unique(mbk.labels_)

ax = fig.add_subplot(1, 3, 3)
for this_centroid, k, col in zip(mbk.cluster_centers_,
                                 range(n_clusters), colors_):
    mask = mbk.labels_ == k
    ax.scatter(X[mask, 0], X[mask, 1], marker='.',
               c='w', edgecolor=col, alpha=0.5)
    ax.scatter(this_centroid[0], this_centroid[1], marker='+',
               c='k', s=25)
ax.set_xlim([-25, 25])
ax.set_ylim([-25, 25])
ax.set_title("MiniBatchKMeans")
ax.set_autoscaley_on(False)
plt.show()

In [ ]:
rng = np.random.RandomState(0)
indices = np.arange(data.trainX.nrows)
rng.shuffle(indices)
unlabeled_index = indices[1000:]

X = data.trainX[:]
X_ = X.reshape(X.shape[0], -1)
Y = data.trainY[:]
Y_train = np.copy(Y)
Y_train[unlabeled_index] = -1

In [ ]:
X_.shape

In [ ]:
cls = label_propagation.LabelSpreading(max_iter=100, kernel='rbf', gamma=0.1)
cls.fit(X_[:10000], Y_train[:10000])
print("Accuracy:%f" % cls.score(X_[unlabeled_index], Y[unlabeled_index]))

In [ ]:
# 设置birch函数
birch = Birch(n_clusters=10)
# 训练数据
y_pred = birch.fit_predict(X_)

In [ ]:
batch_size = 32


In [ ]:
kmeans = MiniBatchKMeans(n_clusters=10, random_state=rng, verbose=True)
##训练数据
y_pred = kmeans.fit(X_)

In [ ]:
np.unique(y_pred.labels_)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn import datasets


def load_data():
    digits=datasets.load_digits()
    rng=np.random.RandomState(0)
    index=np.arange(len(digits.data))
    rng.shuffle(index)
    X=digits.data[index]
    Y=digits.target[index]
    n_labeled_points=int(len(Y)/10)
    unlabeled_index=np.arange(len(Y))[n_labeled_points:]

    return X,Y,unlabeled_index

def test_LabelPropagation(*data):
    X,Y,unlabeled_index=data
    Y_train=np.copy(Y)
    Y_train[unlabeled_index]=-1
    cls=LabelPropagation(max_iter=100,kernel='rbf',gamma=0.1)
    cls.fit(X,Y_train)
    print("Accuracy:%f"%cls.score(X[unlabeled_index],Y[unlabeled_index]))

X,Y,unlabeled_index=load_data()
test_LabelPropagation(X,Y,unlabeled_index)

NameError: name 'LabelPropagation' is not defined

In [ ]:

n_total_samples = len(y)
n_labeled_points = 10
max_iterations = 5

unlabeled_indices = np.arange(n_total_samples)[n_labeled_points:]
f = plt.figure()

for i in range(max_iterations):
    if len(unlabeled_indices) == 0:
        print("No unlabeled items left to label.")
        break
    y_train = np.copy(y)
    y_train[unlabeled_indices] = -1

    lp_model = label_propagation.LabelSpreading(gamma=0.25, max_iter=5)
    lp_model.fit(X, y_train)

    predicted_labels = lp_model.transduction_[unlabeled_indices]
    true_labels = y[unlabeled_indices]

    cm = confusion_matrix(true_labels, predicted_labels,
                          labels=lp_model.classes_)

    print("Iteration %i %s" % (i, 70 * "_"))
    print("Label Spreading model: %d labeled & %d unlabeled (%d total)"
          % (n_labeled_points, n_total_samples - n_labeled_points, n_total_samples))

    print(classification_report(true_labels, predicted_labels))

    print("Confusion matrix")
    print(cm)

    # compute the entropies of transduced label distributions
    pred_entropies = stats.distributions.entropy(
        lp_model.label_distributions_.T)

    # select up to 5 digit examples that the classifier is most uncertain about
    uncertainty_index = np.argsort(pred_entropies)[::-1]
    uncertainty_index = uncertainty_index[
        np.in1d(uncertainty_index, unlabeled_indices)][:5]

    # keep track of indices that we get labels for
    delete_indices = np.array([])

    # for more than 5 iterations, visualize the gain only on the first 5
    if i < 5:
        f.text(.05, (1 - (i + 1) * .183),
               "model %d\n\nfit with\n%d labels" %
               ((i + 1), i * 5 + 10), size=10)
    for index, image_index in enumerate(uncertainty_index):
        image = images[image_index]

        # for more than 5 iterations, visualize the gain only on the first 5
        if i < 5:
            sub = f.add_subplot(5, 5, index + 1 + (5 * i))
            sub.imshow(image, cmap=plt.cm.gray_r)
            sub.set_title("predict: %i\ntrue: %i" % (
                lp_model.transduction_[image_index], y[image_index]), size=10)
            sub.axis('off')

        # labeling 5 points, remote from labeled set
        delete_index, = np.where(unlabeled_indices == image_index)
        delete_indices = np.concatenate((delete_indices, delete_index))

    unlabeled_indices = np.delete(unlabeled_indices, delete_indices)
    n_labeled_points += len(uncertainty_index)

f.suptitle("Active learning with Label Propagation.\nRows show 5 most "
           "uncertain labels to learn with the next model.")
plt.subplots_adjust(0.12, 0.03, 0.9, 0.8, 0.2, 0.45)
plt.show()